## Challenge 0: Install dependencies

In [ ]:
! conda install 'ase==3.24.0' --yes

## Challenge 1: Build a forcefield based equation of state maker

Using `atomate2.common.flows.eos.CommonEosMaker` as a guide, or starting from scratch, write an equation of state workflow for any of the MLIPs available in `atomate2`, or any ASE classical forcefield.

Key questions:
- How do we ensure the EOS is run at fixed volume?
- What should the inputs to the maker look like?


<details>
<summary><b>Hint</b></summary>
<br>
There are MLIP EOS makers in `atomate2` - you can refer to these for guidance.
</details>

### When you are confident in your workflow:
Run it localy (`run_locally`) for the Materials Project structure below ([mp-22526](https://next-gen.materialsproject.org/materials/mp-22526)):


<details>
<summary><b>Reminder</b></summary>
<br>
When you define "complex" (usually mutable) objects as defaults in a python `dataclass`, you need to use the `field` function to define them.
For example, you might see a dataclass like this:

```python
from dataclasses import dataclass, field
@dataclass
class purplePeopleEater:

    eyes : int = 1
    horns : str = "single"
    abilities : dict[str,str] = field(default_factory = dict)
```
which means that a new "instance" of `purplePeopleEater()` will have default attributes of `eyes = 1`, `horns = "single"` and an empty dictionary of abilities.
The line:

```python
abilities : dict[str,str] = field(default_factory = dict)
```
indicates that when the class is called as `purplePeopleEater()`, the field will be given an empty dictionary as default.
</details>

In [7]:
from pymatgen.core import Structure

test_structure = Structure.from_str(
"""mp-22526
1.0
   2.6937121874714256    0.0000000000000000    4.1079559895387510
   1.2267470851777738    2.3981611576247186    4.1079559895387510
   0.0000000000000000    0.0000000000000000    4.9123704799999999
Li Co O
1 1 2
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 Li
   0.5000000000000000    0.5000000000000000    0.5000000000000000 Co
   0.7599932000000000    0.7599932000000000    0.7599932000000000 O
   0.2400068000000000    0.2400068000000000    0.2400068000000000 O
""",
fmt = "poscar"
)

In [ ]:
from __future__ import annotations
from dataclasses import dataclass, field

from jobflow import Maker, Flow, run_locally

@dataclass
class ForceFieldEos(Maker):
    name : str # this is required!
    """
    Fill out any other fields you might need here.
    """
    def make(
        structure : Structure,
        prev_dir : str | None = None # this means that prev_dir can be a string, or None, but defaults to None
    ) -> Flow: # this means that `make` returns a `jobflow` `Flow`
        
        jobs = []

        return Flow(jobs)
    
flow = ForceFieldEos().make(test_structure)
response = run_locally(flow)

### Challenge 1a: Build a postprocessing job for the EOS flow

Using `pymatgen.analysis.eos.EOS` and `matplotlib`, write a function that can be used to fit and plot the output of the EOS flow. How could you incorporate this into the flow as a job?

## Challenge 2: Build a custom relax maker using ASE

Just like `pymatgen`, ASE also has an interface to VASP, among other electronic structure codes.
You can get a sense for how to implement a VASP maker using ASE by using the much simpler EMT calculator from ASE.
This is basically a low-accuracy interatomic potential for alloys/intermetallics.
Build a simple EMT relax maker using `atomate2.ase.jobs.AseMaker`.

The structure of `AseMaker` requires a `run_ase` function and a `calculator` attribute to be defined.
We've defined the `run_ase` function for you, but you still need to call it from `make`.

When you're ready to run it, `run_locally` on the intermetallic structure below ([mp-1228912](https://next-gen.materialsproject.org/materials/mp-1228912)):

In [9]:
intermetallic = Structure.from_str("""mp-1228912
1.0
   3.1077110000000001    0.0000000000000000    0.0000000000000000
   0.0000000000000000    3.1077110000000001    0.0000000000000000
   0.0000000000000000    0.0000000000000000    5.7859559999999997
Al Cu Pd
1 1 2
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 Al
   0.0000000000000000    0.0000000000000000    0.5000000000000000 Cu
   0.5000000000000000    0.5000000000000000    0.2450020000000000 Pd
   0.5000000000000000    0.5000000000000000    0.7549979999999999 Pd
""",
fmt="poscar"
)

In [ ]:
from __future__ import annotations
from dataclasses import dataclass, field
from jobflow import job, Job

from ase.calculators.calculator import Calculator
from ase.calculators.emt import EMT

from atomate2.ase.schemas import AseResult
from atomate2.ase.jobs import AseMaker

from pymatgen.io.ase import AseAtomsAdaptor

@dataclass
class AseEMT(AseMaker):
    name : str # this is required!
    """
    Fill out any other fields you might need here.
    """

    def calculator(self) -> Calculator:
        """ASE EMT calculator."""
        return EMT()
    
    @job
    def make(
        self,
        structure : Structure,
        prev_dir : str | None = None,
    ) -> Job:
        
        return
 
    def run_ase(
        self,
        structure: Structure,
        prev_dir: str | None = None,
    ) -> AseResult:
        
        adaptor = AseAtomsAdaptor()
        atoms = adaptor.get_atoms(structure)
        atoms.calc = self.calculator()
        toten = atoms.get_potential_energy()

        final_structure = adaptor.get_structure(atoms)

        return {
            "final_structure": final_structure,
            "final_total_energy": toten,
        }


### Challenge 2a: Build an EMT EOS maker

Adapt your EOS flow to use the EMT relax maker you just developed.

### Challenge 2b: Modifying the output of the EMT job

What other useful information could be included in either the output of `run_ase` or `make`?
Modify the output of your EMT maker to include the forces or other useful information.